<a href="https://colab.research.google.com/github/Sinan6910/Sinan-java-/blob/main/Untitled14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import os

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [7]:
!cp -r "..//content/gdrive/MyDrive/ses2" ./

In [8]:
data_directory = "/content/gdrive/MyDrive/ses2"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [9]:
audio_path

'/content/gdrive/MyDrive/ses2/audio'

In [10]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15

In [11]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:

            continue
        elif folder in ["other", "_background_noise_"]:

            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

In [12]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [13]:
noise_paths

['/content/gdrive/MyDrive/ses2/noise/other/exercise_bike.wav',
 '/content/gdrive/MyDrive/ses2/noise/other/pink_noise.wav',
 '/content/gdrive/MyDrive/ses2/noise/_background_noise_/running_tap.wav',
 '/content/gdrive/MyDrive/ses2/noise/_background_noise_/10convert.com_Audience-Claps_daSG5fwdA7o.wav',
 '/content/gdrive/MyDrive/ses2/noise/_background_noise_/dude_miaowing.wav',
 '/content/gdrive/MyDrive/ses2/noise/_background_noise_/doing_the_dishes.wav']

In [14]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [15]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

In [16]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [17]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

In [18]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale

    return audio

In [19]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [20]:

class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['ufuk', 'ahmet', 'sinan']
Speaker: ufuk
Speaker: ahmet
Speaker: sinan


In [21]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [22]:
# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

In [23]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [24]:
# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=scale), y),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [25]:
from tensorflow.keras.layers import Conv1D

In [26]:
def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = keras.layers.Conv1D(filters, 1, padding = "same")(x)

    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
        x = keras.layers.Activation(activation)(x)

    x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)

    return keras.layers.MaxPool1D(pool_size = 2, strides = 2)(x)

def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape = input_shape, name = "input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(inputs, 32, 2)
    x = residual_block(inputs, 64, 3)
    x = residual_block(inputs, 128, 3)
    x = residual_block(inputs, 128, 3)
    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output")(x)

    return keras.models.Model(inputs = inputs, outputs = outputs)

model = build_model((sample_rate // 2, 1), len(class_names))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 8000, 1)]            0         []                            
                                                                                                  
 conv1d_15 (Conv1D)          (None, 8000, 128)            512       ['input[0][0]']               
                                                                                                  
 activation_10 (Activation)  (None, 8000, 128)            0         ['conv1d_15[0][0]']           
                                                                                                  
 conv1d_16 (Conv1D)          (None, 8000, 128)            49280     ['activation_10[0][0]']       
                                                                                              

In [27]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/15
13/13 [==============================] - ETA: 0s - loss: 11.7261 - accuracy: 0.4159 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 297s 22s/step - loss: 11.7261 - accuracy: 0.4159 - val_loss: 0.8433 - val_accuracy: 0.5847
Epoch 2/15
13/13 [==============================] - 274s 20s/step - loss: 0.7724 - accuracy: 0.6460 - val_loss: 0.6532 - val_accuracy: 0.7486
Epoch 3/15
13/13 [==============================] - 273s 20s/step - loss: 0.5519 - accuracy: 0.7650 - val_loss: 0.6664 - val_accuracy: 0.7650
Epoch 4/15
13/13 [==============================] - 280s 20s/step - loss: 0.4341 - accuracy: 0.8349 - val_loss: 0.5434 - val_accuracy: 0.7650
Epoch 5/15
13/13 [==============================] - 280s 21s/step - loss: 0.3263 - accuracy: 0.8786 - val_loss: 0.6185 - val_accuracy: 0.7814
Epoch 6/15
13/13 [==============================] - 285s 21s/step - loss: 0.2967 - accuracy: 0.8865 - val_loss: 0.7079 - val_accuracy: 0.7705
Epoch 7/15
13/13 [==============================] - 273s 21s/step - loss: 0.2745 - accuracy: 0.8925 - val_loss: 0.4168 - val_accuracy: 0.8361
Epoch 8/15
13/13

In [28]:
print("Accuracy of model:",model.evaluate(valid_ds))

6/6 [==============================] - 19s 3s/step - loss: 0.3602 - accuracy: 0.8798
Accuracy of model: [0.36016324162483215, 0.8797814249992371]


In [29]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, batch_size, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

4/4 [==============================] - 6s 1s/step
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: sinan	Predicted: sinan
Welcome
The speaker is sinan
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: ufuk	Predicted: sinan
Sorry
 sinan
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: ufuk	Predicted: ufuk
Welcome
The speaker is ufuk
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: ahmet	Predicted: ahmet
Welcome
The speaker is ahmet
Speaker: sinan	Predicted: sinan
Welcome
The speaker is sinan


In [30]:
 def paths_to_dataset(audio_paths):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path, labels):
    test = paths_and_labels_to_dataset(path, labels)


    test = test.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
    )
    test = test.prefetch(tf.data.experimental.AUTOTUNE)


    test = test.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

    for audios, labels in test.take(1):
        ffts = audio_to_fft(audios)
        y_pred = model.predict(ffts)
        rnd = np.random.randint(0, 1, 1)
        audios = audios.numpy()[rnd, :]
        labels = labels.numpy()[rnd]
        y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(1):
            print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
            "[92m",y_pred[index],
                "[92m", y_pred[index]
                )
            )

            print("Speaker Predicted:",class_names[y_pred[index]])



In [46]:
path = ["/content/gdrive/MyDrive/deneme2.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except Exception as e:
    print(f"Error! Check if the file correctly passed or not! {e}")



1/1 [==============================] - 0s 121ms/step
Speaker: 2	Predicted: 2
Speaker Predicted: sinan


In [33]:
# Modeli Google Drive'a kaydedin
model_save_filename = "/content/gdrive/MyDrive/16000/model.h5"
model.save(model_save_filename)


In [47]:
# Modeli GitHub'a yükleme
!git add my_model.h5
!git commit -m "Add trained model"
!git push origin master


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [36]:
# Save the weights
model.save_weights('/content/gdrive/MyDrive/16000/model.h5')

# Restore the weights
model.load_weights('/content/gdrive/MyDrive/16000/model.h5')

# Evaluate the model
test_loss, test_acc = model.evaluate(valid_ds)
print("Test accuracy:", test_acc)


6/6 [==============================] - 20s 3s/step - loss: 0.3602 - accuracy: 0.8798
Test accuracy: 0.8797814249992371
